In [1]:
import ee
import time
import geemap
import ipywidgets as widgets
import tkinter.messagebox

In [4]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=yZLcc9SzVZjOXER8jOehgq0JticzH4RQQ4uWLufSiDc&tc=ltCuT3TltaRvAWbWmBMdJg_vB6A1J6mRIh-0H6oNqlc&cc=Upf5EyWdvVBEox4iNgvDlKLkZ_uy3EbetABWfiitAhI

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXmkcD83EEnZVh3XudAwXNAasF9RPKQu1fAreiommw-Ur33mpS1ozRM

Successfully saved authorization token.


In [5]:
## Change LatLong according to study area
latitude = 16.862758
longitude = -99.839587

In [6]:
## function to mask clouds
def maskClouds(image):
    # Select the QA60 band
    QA60 = image.select(['QA60'])

    # Bits 10 and 11 are cloud and cirrus bits, set all others to 0
    cloud_mask = QA60.bitwiseAnd(1 << 10).eq(0).And(QA60.bitwiseAnd(1 << 11).eq(0))

    # Update the image mask to exclude cloudy and shadowed pixels
    return image.updateMask(cloud_mask)

In [7]:
Map = geemap.Map(center=[latitude, longitude], zoom=11)

# Add Sentinel-2 imagery to the map
# collection = ee.ImageCollection('COPERNICUS/S2')
# image = collection.first()
# Map.addLayer(image, {
#     'bands': ['B4', 'B3', 'B2'],
#     'min': 0,
#     'max': 2000
# }, 'Sentinel-2 Image')

# Create a polygon drawing tool
draw_control = Map.draw_last_feature

# Wait for the user to draw a polygon
## draw_control = Map.draw_last_feature
## draw_control.geometry
print('Please draw a rectangle of the region of interest of your choice using the drawing tool on the left panel')
## export DISPLAY=:0.0
## tkinter.messagebox.showinfo('TITLE', 'Please draw a rectangle if the region of interest of your choice using the drawing tool on the left panel')

# Display the map
Map

Please draw a rectangle of the region of interest of your choice using the drawing tool on the left panel


Map(center=[16.862758, -99.839587], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sea…

In [8]:
draw_control = Map.draw_last_feature
geom = draw_control.geometry()
roi = geom
type(geom)

ee.geometry.Geometry

In [ ]:
# define two dates --> (begin-end)

In [9]:
ee.Initialize()

# Define the region of interest (ROI) using a geometry (e.g., a point, polygon, or feature collection)
## roi = ee.Geometry.Point([longitude, latitude]).buffer(7500)  # Example: Create a buffer around a point

# Define the date range for the image acquisition
start_date_a, end_date_a = '2023-10-01', '2023-10-23'
start_date_b, end_date_b = '2023-10-25', '2023-11-03'

bands = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12']

# Select the Sentinel-2 image collection and filter it based on the ROI and date range
sentinel2_collection = ee.ImageCollection('COPERNICUS/S2_SR')
a_collection = sentinel2_collection.filterBounds(roi).filterDate(start_date_a, end_date_a).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',50)).map(maskClouds)
b_collection = sentinel2_collection.filterBounds(roi).filterDate(start_date_b, end_date_b).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',50)).map(maskClouds)


# compute the geoMedian to obtain a clearer image
reducer = ee.Reducer.geometricMedian(12,0.0000001,500)
geoMedian_a = a_collection.select(bands).reduce(reducer).rename(bands)
geoMedian_b = b_collection.select(bands).reduce(reducer).rename(bands)

# Define export parameters
export_params = [
    #Img antes
     {
      'image': geoMedian_a.toInt16(), ## to keep the size of the image low
      'description': 'geoMedian_acapulco_export_by_user_before',
      'scale': 10,  # Spatial resolution in meters
      'region': roi,
      # maxPixels: 1e12 # You may need this parameter if the roi is too large
      'fileFormat': 'GeoTIFF',
    },
    #Img despues
     {
      'image': geoMedian_b.toInt16(), ## to keep the size of the image low
      'description': 'geoMedian_acapulco_export_by_user_after',
      'scale': 10,  # Spatial resolution in meters
      'region': roi,
      # maxPixels: 1e12 # You may need this parameter if the roi is too large
      'fileFormat': 'GeoTIFF',
    }
]

for param in export_params:
    # Export the image to Google Drive
    task = ee.batch.Export.image.toDrive(**param)
    task.start()

# Monitor the task status
while task.status()['state'] in ['READY', 'RUNNING']:
    print('Task is', task.status()['state'])
    time.sleep(30)  # Optional: Add a delay to reduce API calls frequency

# Check if the export was successful
if task.status()['state'] == 'COMPLETED':
    print('Export completed successfully. Check your Google Drive folder.')
else:
    print('Export failed: ', task.status()['state'])

Task is READY
Task is RUNNING
Task is RUNNING
Task is RUNNING
Task is RUNNING
Task is RUNNING
Task is RUNNING
Task is RUNNING
Task is RUNNING
Task is RUNNING
Export completed successfully. Check your Google Drive folder.
